In [1]:
import math
import time
import pandas as pd

## Load precomputed combinations to save computation time

In [ ]:
comb_df = pd.read_pickle("precompute_combinations/precomputed_combinations_df.pkl")

def choose(n, k):
    return comb_df.loc[(comb_df["n"] == n) & (comb_df["k"] == k), "n_choose_k"].values[0]

## Load probabilities

In [2]:
# Approximate probability of a goal in a given second
seconds_in_game = 20 * 60 * 3
p_a1 = 3.12 / seconds_in_game
p_b1 = p_a1

# Probability of the other team scoring during 6v5 increases
p_b2 = 1.5 * p_b1

# Probability of our team scoring during 6v5 also increases, though not as much
p_a1 = 1.25 * p_a1

# Let's say the current score is 1 - 3
a_0 = 1
b_0 = 3

# 5 minutes left in the game
T = 5 * 60

In [ ]:
decisions_df = pd.DataFrame(columns = [
    'pull_time',
    'prob_success',
    'compute_time'])

#  Try out each possible time to pull the goalie
for i in range(0, int(T/10)):
    s = i * 10
    start_time = time.time()

    prob_success = 0

    # Number of goals scored by other team before pulling the goalie
    for g_b1 in range(0, s):
        prob_gb1 = choose(s, g_b1) * (p_b1 ** g_b1) * ((1 - p_b1) ** (s - g_b1))

        # Number of goals scored by our team before pulling the goalie
        for g_a1 in range(0, s):
            prob_ga1 = choose(s, g_a1) * (p_a1 ** g_a1) * ((1 - p_a1) ** (s - g_a1))

            # Number of goals scored by other team after pulling the goalie
            for g_b2 in range(0, T - s):
                prob_gb2 = choose(T - s, g_b2) * (p_b2 ** g_b2) * ((1 - p_b2) ** (T - s - g_b2))

                # Number of goals scored by our team after pulling the goalie
                for g_a2 in range(0, T - s):
                    prob_ga2 = choose(T - s, g_a2) * (p_a1 ** g_a2) * ((1 - p_a1) ** (T - s - g_a2))

                    # Probability of this event
                    prob = prob_gb1 * prob_ga1 * prob_gb2 * prob_ga2

                    # Update the score
                    a = a_0 + g_a1 + g_a2
                    b = b_0 + g_b1 + g_b2

                    # Success if we win or tie
                    success = (a >= b)

                    prob_success = prob_success + prob * success
    end_time = time.time()
    elapsed_time = end_time - start_time

    decisions_df = pd.concat(
        [
            decisions_df,
            pd.DataFrame({
                "pull_time": s,
                "prob_success": prob_success,
                "compute_time": elapsed_time
            }, index=[0]),
        ], ignore_index=True
    )

display(decisions_df)

'Choosing to pull the goalie at 0 seconds gives a 0.00% chance of success (took 0.00 seconds to compute)'
'Choosing to pull the goalie at 10 seconds gives a 3.01% chance of success (took 15.67 seconds to compute)'
'Choosing to pull the goalie at 20 seconds gives a 3.02% chance of success (took 54.81 seconds to compute)'
'Choosing to pull the goalie at 30 seconds gives a 3.04% chance of success (took 107.84 seconds to compute)'
'Choosing to pull the goalie at 40 seconds gives a 3.05% chance of success (took 177.16 seconds to compute)'
'Choosing to pull the goalie at 60 seconds gives a 3.07% chance of success (took 295.79 seconds to compute)'
'Choosing to pull the goalie at 70 seconds gives a 3.08% chance of success (took 357.98 seconds to compute)'
'Choosing to pull the goalie at 150 seconds gives a 3.18% chance of success (took 425.45 seconds to compute)'